# **Solutions**
https://github.com/Clement25/Speech-and-Language-Processing-ver3-solutions

# **Cap 3**

3.1 creation of N gram

In [ ]:
def prepro_text(text):
  text = text.strip().split(' ')
  text = [n for n in text if n != ' ']

  if text[0] != '<s>': text.insert(0, '<s>')
  if text[-1] != '</s>': text.insert(len(text), '</s>')
  
  return text

def add_one(dic, key):
  if key in dic.keys():
    dic[key] += 1
  else:
    dic[key] = 1
  return dic

def create_ngram(text, i, N):
  combinacion = []
  for n in reversed(range(1, N)):
    token = [text[i-n] if i-n > -1 else '<s>'][0]
    combinacion.append(token)
  combinacion = ' '.join(combinacion)
  return combinacion

def n_gram(w_h, text):
  text = prepro_text(text)

  w_h = w_h.strip().split(' ')
  N = len(w_h)+1

  palabras_antes = {}
  palabras_proba = {}

  combinacion = []
  for i, x in enumerate(text):
    combinacion = create_ngram(text, i, N)
    if w_h == combinacion.split(' '):
      palabras_antes = add_one(palabras_antes, x)

  for pal in palabras_antes:
    numerador = palabras_antes[pal]
    denominador = text.count(pal)
    palabras_proba[pal] = numerador/denominador
  try:
    return max(palabras_proba, key=palabras_proba.get)
  except:
    return '</s>'

In [ ]:
text = 'me llamo Felipe porque sandia no es sandia no es es y no me llamo Felipe pero se llama Felipe pero no me llamo Jose pero no se llama Jose llama Jose llama Jose y sandia es'
w_h = 'sandia'

for i in range(15):
  print(w_h)
  w_h = n_gram(w_h, text)

sandia
no
me
llamo
Felipe
porque
sandia
no
me
llamo
Felipe
porque
sandia
no
me


3.2

In [ ]:
i_start = 0.25
want_i = 0.33
chinese_want = 0.0065
food_chinese = 0.52
end_food = 0.68

res_uno = i_start * want_i * chinese_want * food_chinese * end_food * 1000

In [ ]:
def add_one_smooth(wn_1, w1, v):
  return ((wn_1 + 1) * w1) / (w1 + v)

3.4

In [ ]:
def flatten(text):
  new = ''
  for n in text:
    new += n+' '
  return new

In [ ]:
def bigram(w_n, text, smooth = False):

  if smooth:
    add = 1
    V = len(set(prepro_text(flatten(text))))
  else:
    add = 0
    V = 0

  palabras_antes = {}
  palabras_proba = {}

  for sent in text:
    sent = prepro_text(sent)
    for i in range(len(sent)-1):
      if w_n == sent[i]:
        palabras_antes = add_one(palabras_antes, sent[i+1])

  for pal in palabras_antes:
    numerador = palabras_antes[pal]
    denominador = prepro_text(flatten(text)).count(w_n)
    palabras_proba[pal] = (numerador + add)/(denominador + V)
  try:
    result = max(palabras_proba, key=palabras_proba.get)
    print(f'Prob: {palabras_proba[result]}')
    return result
  except:
    return '</s>'

In [ ]:
text = ['<s> I am Sam </s>','<s> Sam I am </s>', '<s> I am Sam </s>', '<s> I do not like green eggs and Sam </s>']
bigram('green', text, smooth = True)

Prob: 0.16666666666666666


'eggs'

In [ ]:
def add1smooth(text, pre, act):

  V = len(set(prepro_text(flatten(text))))
  N = 0
  match = 0

  for sent in text:
    sent = prepro_text(sent)
    for i in range(len(sent)-1):
      if sent[i] == pre:
        N += 1
        if sent[i+1] == act:
          match += 1

  return (match + 1) / (N + V)

In [ ]:
text = ['<s> I am Sam </s>','<s> Sam I am </s>', '<s> I am Sam </s>', '<s> I do not like green eggs and Sam </s>']
add1smooth(text, 'green', 'eggs')

0.16666666666666666

3.5

In [ ]:
text = ['<s> a b', '<s> b b', '<s> b a', '<s> a a']

combinations = [['a', 'b'], ['b', 'a'], ['a', 'a'], ['b', 'b']]

sumar = 0
for c in combinations:
  sumar += add1smooth(text, c[0], c[1])

sumar

1.0

3.6

In [ ]:
# import collections
def unigram(sents):
    ug_count_dic = collections.defaultdict(int)
    ug_prob_dic = collections.defaultdict(float)
    total = len(prepro_text(flatten(text)))
    for sent in sents:
        for word in sent.split(' '):
            if word not in ug_count_dic:
                ug_count_dic[word] = 1
            else:
                ug_count_dic[word] += 1
        for word in ug_count_dic:
            ug_prob_dic[word] = ug_count_dic[word]/total
    return ug_count_dic, ug_prob_dic

3.11

In [ ]:
def bigrams(sents, ug_count_dic):
    bg_dic = collections.defaultdict(float)
    for sen in sents:
        sen = sen.split(' ')
        for i in range(len(sen)-1):
            if (sen[i], sen[i+1]) not in bg_dic:
                bg_dic[(sen[i],sen[i+1])] = 1
            else:
                bg_dic[(sen[i],sen[i+1])] += 1
    for bi in bg_dic:
        bg_dic[bi] = bg_dic[bi]/ug_count_dic[bi[0]]
    return bg_dic

In [ ]:
text = ['<s> I am Sam </s>','<s> Sam I am </s>', '<s> I am Sam </s>', '<s> I do not like green eggs and Sam </s>']
ug_count_dic, ug_prob_dic = unigram(text)
bi_dic = bigrams(text, ug_count_dic)

defaultdict(<class 'float'>, {('<s>', 'I'): 3, ('I', 'am'): 3, ('am', 'Sam'): 2, ('Sam', '</s>'): 3, ('<s>', 'Sam'): 1, ('Sam', 'I'): 1, ('am', '</s>'): 1, ('I', 'do'): 1, ('do', 'not'): 1, ('not', 'like'): 1, ('like', 'green'): 1, ('green', 'eggs'): 1, ('eggs', 'and'): 1, ('and', 'Sam'): 1})


In [ ]:
ug_sorted_dic = {x:v for x,v in sorted(ug_prob_dic.items(), key=lambda item:item[1], reverse=True)}
print(ug_sorted_dic)

{'<s>': 0.16, 'I': 0.16, 'Sam': 0.16, '</s>': 0.16, 'am': 0.12, 'do': 0.04, 'not': 0.04, 'like': 0.04, 'green': 0.04, 'eggs': 0.04, 'and': 0.04}


In [ ]:
bi_sorted_dic = {x:v for x,v in sorted(bi_dic.items(), key=lambda item:item[1], reverse=True)}
print(bi_sorted_dic)

{('do', 'not'): 1.0, ('not', 'like'): 1.0, ('like', 'green'): 1.0, ('green', 'eggs'): 1.0, ('eggs', 'and'): 1.0, ('and', 'Sam'): 1.0, ('<s>', 'I'): 0.75, ('I', 'am'): 0.75, ('Sam', '</s>'): 0.75, ('am', 'Sam'): 0.6666666666666666, ('am', '</s>'): 0.3333333333333333, ('<s>', 'Sam'): 0.25, ('Sam', 'I'): 0.25, ('I', 'do'): 0.25}


In [ ]:
def cal_ppl(text, bi_dic):
  sent = text.split(' ')
  ppl = 1
  for i in range(1,len(sent)-2):
      w1, w2 = sent[i], sent[i+1]
      ppl *= bi_dic[(w1,w2)]
  return pow(ppl, -1/(len(sent)-2))

In [ ]:
for t in text:
  print(cal_ppl(t, bi_dic))
  print(t, '\n')

1.2599210498948732
<s> I am Sam </s> 

1.7471609294725976
<s> Sam I am </s> 

1.2599210498948732
<s> I am Sam </s> 

1.189207115002721
<s> I do not like green eggs and Sam </s> 



3.12

In [ ]:
pow((0.91**9)*0.01, -1/10)

1.7252925496828495

# **CAP 4**

4.2

In [ ]:
data = {
    'comedy':[['fun, couple, love, love'], ['couple, fly, fast, fun, fun']],
    'action':[['fast, furious, shoot'], ['furious, shoot, shoot, fun'], ['fly, fast, shoot, love']]
}

D = 'fast, couple, shoot, fly'

In [ ]:
def unnest(lists):
  return list(chain(*lists))

In [ ]:
# Train
from itertools import chain
import math

def Bayesian_train(data, binary = False):

  def binary_bayes_w_count(data, w):
    ocurrances_w = 0
    for d in unnest(data):
        if w in d:
          ocurrances_w += 1
    ocurrances_w += 1
    return ocurrances_w

  clases = list(data.keys())

  Pc = {}
  likelihood = {}

  for c in clases:
    sents = unnest(data.values())

    N = len(sents)
    Nc = len(data[c])

    Pc[c] = math.log10(Nc/N)

    V = set(', '.join(unnest(sents)).split(', '))

    big_doc = ', '.join(unnest(data[c])).split(', ')

    for w in V:
      if binary:
        ocurrances_w = binary_bayes_w_count(data[c], w)
        ocurrances_t = len(unnest(data[c])) + len(V) #este denominador podria estar mal
      else:
        ocurrances_w = big_doc.count(w) +1
        ocurrances_t = sum([1 for w in big_doc if w in V]) + len(V)
      
      p_w_c = math.log10(ocurrances_w/ocurrances_t)

      likelihood[w,c] = p_w_c

      print(f'Assigned {(w,c)} = {p_w_c} because {ocurrances_w} / {ocurrances_t}\n')
    
  return Pc, likelihood, V

In [ ]:
Pc, likelihood, V = Bayesian_train(data)

Assigned ('poor', 'pos') = -0.7781512503836436 because 2 / 12

Assigned ('great', 'pos') = -0.3010299956639812 because 6 / 12

Assigned ('good', 'pos') = -0.4771212547196625 because 4 / 12

Assigned ('poor', 'neg') = -0.18905623622004886 because 11 / 17

Assigned ('great', 'neg') = -0.7533276666586115 because 3 / 17

Assigned ('good', 'neg') = -0.7533276666586115 because 3 / 17



In [ ]:
# Test

def Bayesian_pred(data, likelihood, V, D):

  clases = list(data.keys())
  suma = {}

  for c in clases:
    suma[c] = Pc[c]

    for w in D.split(' '): # cambiar split a comas si es necesario
      if w in V:
        print(f'Summed {likelihood[w,c]} to {c} because {w} in test D.')
        suma[c] += likelihood[w,c]
  print(suma)
  return max(suma, key=suma.get)

In [ ]:
pred = Bayesian_pred(data, likelihood, V, D)
pred

Summed -0.4771212547196625 to pos because good in test D.
Summed -0.4771212547196625 to pos because good in test D.
Summed -0.3010299956639812 to pos because great in test D.
Summed -0.7781512503836436 to pos because poor in test D.
Summed -0.7533276666586115 to neg because good in test D.
Summed -0.7533276666586115 to neg because good in test D.
Summed -0.7533276666586115 to neg because great in test D.
Summed -0.18905623622004886 to neg because poor in test D.
{'pos': -2.431363764158988, 'neg': -2.6708879858122394}


'pos'

4.3

In [ ]:
data = {
    'pos':[['good, good, good, great, great, great'], ['poor, great, great']],
    'neg':[['good, poor, poor, poor'], ['good, poor, poor, poor, poor, poor, great, great'], ['poor, poor']]
}

D = 'a good good plot and great characters but poor acting'

In [ ]:
Pc, likelihood, V = Bayesian_train(data)

Assigned ('poor', 'pos') = -0.7781512503836436 because 2 / 12

Assigned ('great', 'pos') = -0.3010299956639812 because 6 / 12

Assigned ('good', 'pos') = -0.4771212547196625 because 4 / 12

Assigned ('poor', 'neg') = -0.18905623622004886 because 11 / 17

Assigned ('great', 'neg') = -0.7533276666586115 because 3 / 17

Assigned ('good', 'neg') = -0.7533276666586115 because 3 / 17



In [ ]:
pred = Bayesian_pred(data, likelihood, V, D)
pred

Summed -0.4771212547196625 to pos because good in test D.
Summed -0.4771212547196625 to pos because good in test D.
Summed -0.3010299956639812 to pos because great in test D.
Summed -0.7781512503836436 to pos because poor in test D.
Summed -0.7533276666586115 to neg because good in test D.
Summed -0.7533276666586115 to neg because good in test D.
Summed -0.7533276666586115 to neg because great in test D.
Summed -0.18905623622004886 to neg because poor in test D.
{'pos': -2.431363764158988, 'neg': -2.6708879858122394}


'pos'

In [ ]:
# Binary
Pc, likelihood, V = Bayesian_train(data, binary = True)

Assigned ('poor', 'pos') = -0.3979400086720376 because 2 / 5

Assigned ('great', 'pos') = -0.22184874961635637 because 3 / 5

Assigned ('good', 'pos') = -0.3979400086720376 because 2 / 5

Assigned ('poor', 'neg') = -0.17609125905568127 because 4 / 6

Assigned ('great', 'neg') = -0.4771212547196625 because 2 / 6

Assigned ('good', 'neg') = -0.3010299956639812 because 3 / 6



In [ ]:
pred = Bayesian_pred(data, likelihood, V, D)
pred

Summed -0.3979400086720376 to pos because good in test D.
Summed -0.3979400086720376 to pos because good in test D.
Summed -0.22184874961635637 to pos because great in test D.
Summed -0.3979400086720376 to pos because poor in test D.
Summed -0.3010299956639812 to neg because good in test D.
Summed -0.3010299956639812 to neg because good in test D.
Summed -0.4771212547196625 to neg because great in test D.
Summed -0.17609125905568127 to neg because poor in test D.
{'pos': -1.813608784304507, 'neg': -1.4771212547196626}


'neg'

# **CAP 5**

5.1

In [ ]:
import math

In [ ]:
200**0.75

53.182958969449885

# **CAP 8**

8.1

In [ ]:
# Imports
import nltk
from sklearn.model_selection import train_test_split
import time

In [ ]:
#loading corpus
nltk.download('treebank')
tagged_sentences = nltk.corpus.treebank.tagged_sents()

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


In [ ]:
#Splitting test and train dataset
train , test = train_test_split(tagged_sentences, shuffle=True, test_size=0.2) 

In [ ]:
# Baseline Tagger
class BaselineTagger():
  def __init__(self):
    pass
  
  def train_bl(self, train_pairs):
    print(f'Training with {len(train_pairs)} sentences.\n')
    start = time.time()
    self.word_dic = word_dic = {}
    for sent in train_pairs:
      for word, tag in sent:
        if word not in self.word_dic:
          self.word_dic[word] = {tag:1}
        else:
          self.word_dic[word] = add_one(self.word_dic[word], tag)
    print(f'Training finished. It took {round((time.time()-start),3)} seconds.\n')

  def predict_bl(self, word):
    if word in self.word_dic:
      result = max(self.word_dic[word], key=self.word_dic[word].get)
    else:
      result = 'NN'
    return result
  
  def test_bl(self, test_pairs):
    score = 0
    count = 0
    for sent in test_pairs:
      for word, tag in sent:
        count += 1
        prediction = self.predict(word)
        gold_label = tag
        score += (prediction == gold_label)
        if count%500 == 0:print(f'Acc: {round((score/count),3)}, {count} words tested.\n')

  # Helper
  def add_one(self, dic, key):
    if key in dic.keys():
      dic[key] += 1
    else:
      dic[key] = 1
    return dic

In [ ]:
# Test
bl = BaselineTagger()
bl.train(train)
bl.test(test)

8.5

Hidden Markov Model (HMM) and Viterbi Algorithm

In [ ]:
class HMM(BaselineTagger):
  def __init__(self):
    pass
  
  def train_hmm(self, train_set):
    print(f'Training with {len(train_set)} sentences.\n')
    start = time.time()
    self.train_set = train_set
    self.starting_prob = self.build_C()
    self.transition_prob = self.build_A()
    self.obs_lh = self.build_B()
    self.bl = BaselineTagger() # baseline_tagger for if all p = 0
    self.bl.train_bl(self.train_set)
    print(f'Training finished. It took {round((time.time()-start),3)} seconds.\n')
    
  def build_C(self): # Starting prob (C)
    starting_prob = {}
    for sent in self.train_set:
      word, tag = sent[0]
      starting_prob = self.add_one(starting_prob, tag)
    # Normalising
    total = sum(starting_prob.values())
    starting_prob = {k: v / total for k, v in starting_prob.items()}
    return starting_prob

  def build_A(self): # Transition prob (A)
      transition_prob = {}
      for sent in self.train_set:
        for i, (word, tag) in enumerate(sent[:-1]):
          next_tag = sent[+1][-1]
          if tag not in transition_prob:
            transition_prob[tag] = {next_tag:1}
          else:
            transition_prob[tag] = self.add_one(transition_prob[tag], next_tag)
      # Normalising
      for key in transition_prob:
        total = sum(transition_prob[key].values())
        transition_prob[key] = {k: v / total for k, v in transition_prob[key].items()}
      return transition_prob
  
  def build_B(self): # Observation Likelihood (B)
    obs_lh = {}
    for sent in self.train_set:
      for word, tag in sent:
        if tag not in obs_lh:
          obs_lh[tag] = {word:1}
        else:
          obs_lh[tag] = self.add_one(obs_lh[tag], word)
    # Normalising
    for key in obs_lh:
      total = sum(obs_lh[key].values())
      obs_lh[key] = {k: v / total for k, v in obs_lh[key].items()}
    return obs_lh

  
  def predict_hmm(self, test_set):
    score = 0
    count = 0

    for ind, sent in enumerate(test_set):
      N = len(self.starting_prob)
      T = len(sent)
      matrix = np.zeros((N,T))
      possible_tags = []

      # First column
      first_column = []
      first_word = sent[0][0]
      for key, value in self.starting_prob.items():
        possible_tags.append(key)
        if first_word in self.obs_lh[key]:
          first_column.append(value * self.obs_lh[key][first_word])
        else:
          first_column.append(0.0)
      matrix[:,0] = first_column

      # Rest of the matrix
      for t in range(1,T):
        word, tag = sent[t]
        for s in range(N):
          s_tag = possible_tags[s]
          candidates = []
          for s2 in range(N):
            s2_tag = possible_tags[s2]
            if matrix[s2,t-1] > 0.0 and s_tag in self.transition_prob[s2_tag] and word in self.obs_lh[s_tag]:
              candidates.append(matrix[s2,t-1] * self.transition_prob[s2_tag][s_tag] * self.obs_lh[s_tag][word])
            else:
              candidates.append(0.0)
            matrix[s,t] = max(candidates)

      # Prediction
      for col in range(matrix.shape[-1]):
        count += 1
        gold_label = sent[col][-1]

        max_val = max(matrix[:,col])
        max_ind = list(matrix[:,col]).index(max_val)

        if max_val > 0:
          predicted = possible_tags[max_ind]
          score += (predicted == gold_label)
        else:
          predicted = self.bl.predict_bl(sent[col])
        score += (predicted == gold_label)
        
      if count%200==0:print(f'Acc: {round((score/count),3)}, {count} words tested.')

In [ ]:
hmm = HMM()
hmm.train_hmm(train)
hmm.predict_hmm(test)

Training with 3131 sentences.

Training with 3131 sentences.

Training finished. It took 0.147 seconds.

Acc: 0.548, 12000 words tested.
Acc: 0.552, 13000 words tested.
Acc: 0.549, 19500 words tested.
Acc: 0.551, 20000 words tested.


# **CAP 10**
10.1

In [ ]:
gold_standar = 'witness for the past,'
translation = 'past witness'

total_unigrams_produced = 11
total_bigrams_produced = 9

unigram_precition = 11/11
unigram_recall = 11/18

bigram_precition = 9/9
bigram_recall = 9/17

total_precition = (unigram_precition + bigram_precition) / 2
total_recall = (unigram_recall + bigram_recall) / 2


def chrF(B, total_precition, total_recall):
   B = B**2
   return ((B+1) * total_precition * total_recall) / (B * total_precition + total_recall)
                                                      
B = 2
chrF(B, total_precition, total_recall)